In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import subprocess
import sys

In [2]:
curr_dir = !pwd
ligands = ["sm"]
model = "XGB"
datafile_date = "08.06.18"
folds = 5
dirname = "comb_dna0.5_rna0.5_ion0.75"
features_type = "all_features_probs"

oof_path = curr_dir[0]+"/stacked_oof_predictions/"
heldout_fold_path = curr_dir[0]+"/../10.Prediction/stacking/2nd_level_pred/08.06.18_dna0.5_rna0.5_ion0.75/global_auprc/"+features_type+"/per_fold/"
out_path = curr_dir[0]+"/test_oof_combined/"+features_type+"/"

In [3]:
#%%time
pred_prob_dict = defaultdict(dict)

for ligand in ligands:
    #Read the data tables
    
    pred_prob_dict[ligand][model] = dict()

    for heldout_fold in range(1,folds+1):
        pred_prob_dict[ligand][model][heldout_fold] = dict()

        #Reading the ligand-model oof training predictions (of all other 4 folds)
        pred_prob_dict[ligand][model][heldout_fold]["oof-train"] = pd.DataFrame()
        for test_fold in range(1, folds+1):

            if (test_fold == heldout_fold):
                continue
            curr_oof_table = pd.read_csv(oof_path+str(heldout_fold)+"/"+ligand+"_"+model+"_test_fold"+str(test_fold)+"_5w.csv", sep='\t', index_col=0)[["fold", "idx", "prob"]]
            #curr_oof_table.columns = ["idx", "prob"]

            if (pred_prob_dict[ligand][model][heldout_fold]["oof-train"].shape[0] == 0): 
                pred_prob_dict[ligand][model][heldout_fold]["oof-train"] = curr_oof_table
            else:
                pred_prob_dict[ligand][model][heldout_fold]["oof-train"] = pd.concat([pred_prob_dict[ligand][model][heldout_fold]["oof-train"], curr_oof_table])

        #Reading the ligand-model heldout fold predictions
        pred_prob_dict[ligand][model][heldout_fold]["heldout"] = pd.read_csv(heldout_fold_path+ligand+"_"+model+"_fold"+str(heldout_fold)+"_5w.csv", sep='\t', index_col=0)[["fold", "idx", "prob"]]
        #pred_prob_dict[ligand][model][heldout_fold]["heldout"].columns = ["idx", "prob"]
            
               
    print "Finished reading data "+ligand
    
    #Combine them into ligand-folds tables
    for heldout_fold in range(1,folds+1):
        ligand_fold_combined_table = pd.DataFrame()
        
        model_combined_table = pd.concat([pred_prob_dict[ligand][model][heldout_fold]["heldout"], pred_prob_dict[ligand][model][heldout_fold]["oof-train"]])

        if (ligand_fold_combined_table.shape[0] == 0):
            ligand_fold_combined_table = model_combined_table
        else:
            ligand_fold_combined_table = pd.merge(ligand_fold_combined_table, model_combined_table, on="idx")

        #Saving to .csv
        ligand_fold_combined_table = ligand_fold_combined_table.sort_values(by="idx")
        ligand_fold_combined_table = ligand_fold_combined_table.reset_index(drop=True)
        ligand_fold_combined_table.to_csv(out_path+str(heldout_fold)+"/"+ligand+"_stacking_probs.csv", sep='\t')
        
        print "Finished writing "+ligand+" heldout fold: "+str(heldout_fold)

Finished reading data sm
Finished writing sm heldout fold: 1
Finished writing sm heldout fold: 2
Finished writing sm heldout fold: 3
Finished writing sm heldout fold: 4
Finished writing sm heldout fold: 5
